# CrunchBase

This loads the CrunchBase data from Nesta's DAPS system


## Preamble

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
#Basic imports
import pandas as pd
import numpy as np
from data_getters.core import get_engine
from data_getters.inspector import get_schemas
from random import sample


import matplotlib.pyplot as plt

today_str = str(datetime.date.today())

In [ ]:
#We create this type to deal with some Nones later
NoneType = type(None)

## Load data

In [ ]:
#Connection to the database

my_config_here ="../../mysqldb_team.config"

con = get_engine(my_config_here)

In [ ]:
#Read organisations
comps_chunks = pd.read_sql_table('crunchbase_organizations', con, chunksize=1000)

#Read categories
cats_chunks = pd.read_sql_table('crunchbase_organizations_categories', con, chunksize=1000)

#Concatenate the chunks into dfs
comps, cats = [pd.concat(x).reset_index(drop=True) for x in [comps_chunks,cats_chunks]]
#descr_short, descr_long = [comps[v].apply(lambda x: x.lower() if type(x)==str else np.nan) for v in ['short_description','long_description']]

In [ ]:
print(len(comps))

print(len(cats))

print(len(set(cats['organization_id'])))

There are around 50,000 organisations without categories



In [ ]:
comps.head()

In [ ]:
cats.head()

## Data processing

### Combine comps and cats

In [ ]:
#Merge organisations and categories

#We reset the index because you can't merge series
cats_grouped = cats.groupby('organization_id')['category_name'].apply(lambda x: list(x)).reset_index(drop=False)

#This gives us a dataframe with a new field with the list of categories for the organisation
comps_cats = pd.merge(comps,cats_grouped,left_on='id',right_on='organization_id')

In [ ]:
comps_cats.shape

This is as expected given the length of dfs above (there seems to be a small number of organisations in the `cat` df not included in the `comps` df but we can worry about that a bit later.

### Merge comps and locations

Merges companies and locations

In [ ]:
locations = pd.read_sql('geographic_data',con)

In [ ]:
comps_geo = pd.merge(comps_cats,locations,left_on='location_id',right_on='id',suffixes=['','_locs'])

In [ ]:
#Drop repeated columns
comps_geo.drop([x for x in comps_geo.columns if '_locs' in x],axis=1,inplace=True)

In [ ]:
comps_geo.to_csv(f'../../data/external/{today_str}_cb_companies.csv',compression='zip',index=False)